In [92]:
# ! pip install "autogen-ext[openai]"
# ! pip install autogen-agentchat
# ! pip install azure-ai-projects 
# ! pip install azure-identity

In [93]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_agentchat.ui import Console
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool, AgentThread
import os, requests, autogen

In [94]:
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

In [ ]:
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4",
    api_version="2024-12-01-preview",
    model = "gpt-4",
    azure_endpoint="our Azure OpenAI Service endpoint from Azure AI Foundry",
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key="Your API Key from Azure AI Foundry", # For key-based authentication.
)

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str='Your Project connection string from Azure Foundry here',
)

In [ ]:
# Function to fetch news articles from a web search API
def fetch_news(topic):
    search_url = "https://newsapi.org/v2/everything"
    api_key = "XXXXXXXXXXX"  # Replace with your NewsAPI key
    params = {
        "q": topic,
        "apiKey": api_key,
        "language": "en",
        "sortBy": "relevancy"
    }
    response = requests.get(search_url, params=params)
    if response.status_code == 200:
        articles = response.json().get("articles", [])
        return [article["title"] + " - " + article["url"] for article in articles[:5]]
    else:
        return ["Failed to fetch news"]

In [98]:
# Function to handle news request
def get_news(topic):
    news_list = fetch_news(topic)
    news_summary = "\n".join(news_list)
    return news_summary

In [99]:
#thread = AgentThread()

In [100]:
async def generate_news_feed(topic: str) -> str:
    print("This is Code Interpreter for Azure AI Agent Service .......")
    print(topic)
    with project_client:

        code_interpreter = CodeInterpreterTool()
        agent = project_client.agents.create_agent(
            model="gpt-4",
            name="NewsFeed-Assistant",
            instructions="""        
                You are a news feed agent.
                Your work is to collate all the latest news feed you receive on a topic in an order.
                You don't have to visit the links, just collate the news feed and provide the summary.
                You can use the code interpreter to run any code you need to process the news feed.
                You can use the tools provided to you to fetch the news feed.
            """,
            tools=code_interpreter.definitions,
            headers={"x-ms-enable-preview": "true"}
        )
        print(f"Created agent, ID: {agent.id}")
        
        # Create thread for communication
        thread = project_client.agents.create_thread()
        print(f"Created thread, ID: {thread.id}")

        # Create message to thread
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=get_news(topic),
        )
        print(f"SMS: {message}")
        # Create and process agent run in thread with tools
        run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
        print(f"Run finished with status: {run.status}")

        if run.status == "failed":
            print(f"Run failed: {run.last_error}")

        # Delete the assistant when done
        # project_client.agents.delete_agent(agent.id)
        # print("Deleted agent")

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        print("Messages:"+ messages["data"][0]["content"][0]["text"]["value"])
        return messages["data"][0]["content"][0]["text"]["value"]

In [ ]:
# bing_connection = project_client.connections.get(
#     connection_name='Grounding with Bing Search connection name from Azure Foundry here'
# )

# conn_id = bing_connection.id

In [102]:
# async def web_ai_agent(query: str) -> str:
#     print("This is Bing for Azure AI Agent Service .......")
#     bing = BingGroundingTool(connection_id=conn_id)
#     with project_client:
#         agent = project_client.agents.create_agent(
#             model="gpt-4",
#             name="bing-agent",
#             instructions="""        
#                 You are a web search agent.
#                 Your only tool is search_tool - use it to find information.
#                 You make only one search call at a time.
#                 Once you have the results, you never do calculations based on them.
#             """,
#             tools=bing.definitions,
#             headers={"x-ms-enable-preview": "true"}
#         )
#         print(f"Created agent, ID: {agent.id}")

#         # Create thread for communication
#         thread = project_client.agents.create_thread()
#         print(f"Created thread, ID: {thread.id}")

#         # Create message to thread
#         message = project_client.agents.create_message(
#             thread_id=thread.id,
#             role="user",
#             content=query,
#         )
#         print(f"SMS: {message}")
#         # Create and process agent run in thread with tools
#         run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
#         print(f"Run finished with status: {run.status}")

#         if run.status == "failed":
#             print(f"Run failed: {run.last_error}")

#         # Delete the assistant when done
#         # project_client.agents.delete_agent(agent.id)
#         # print("Deleted agent")

#         # Fetch and log all messages
#         messages = project_client.agents.list_messages(thread_id=thread.id)
#         # print("Messages:"+ messages["data"][0]["content"][0]["text"]["value"])
#     return messages["data"][0]["content"][0]["text"]["value"]



In [103]:
async def save_blog_agent(blog_content: str) -> str:

    # print("This is Code Interpreter for Azure AI Agent Service .......")
    # print(blog_content)
    with project_client:

        code_interpreter = CodeInterpreterTool()
        
        agent = project_client.agents.create_agent(
            model="gpt-4o-mini",
            name="save-agent",
            instructions="You are helpful agent",
            tools=code_interpreter.definitions,
            # tool_resources=code_interpreter.resources,
        )
        
        thread = project_client.agents.create_thread()

        message = project_client.agents.create_message(
                thread_id=thread.id,
                role="user",
                content="""
                    You are my Python programming assistant. Generate code and execute it according to the following requirements

                    1. Save """ + blog_content + """ to blog-{YYMMDDHHMMSS}.md

                    2. give me the download this file link
                """,

        )
        # create and execute a run
        run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
        # print(f"Run finished with status: {run.status}")

        if run.status == "failed":
            # Check if you got "Rate limit is exceeded.", then you want to get more quota
            print(f"Run failed: {run.last_error}")

        messages = project_client.agents.list_messages(thread_id=thread.id)

        for file_path_annotation in messages.file_path_annotations:

            file_name = os.path.basename(file_path_annotation.text)

            project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./blog")
            

        #project_client.agents.delete_agent(agent.id)
        #print("Deleted agent")
        return "Saved"


    
    

In [104]:
# bing_search_agent = AssistantAgent(
#     name="bing_search_agent",
#     model_client=az_model_client,
#     tools=[web_ai_agent],
#     system_message="Use tools to solve tasks.",
# )

In [105]:
news_feed_agent = AssistantAgent(
    name="news_feed_agent",
    model_client=az_model_client,
    tools=[generate_news_feed],
    system_message="Use tools to solve tasks.",
)

In [106]:
save_blog_content_agent = AssistantAgent(
    name="save_blog_content_agent",
    model_client=az_model_client,
    tools=[save_blog_agent],
    system_message="""
        Save blog content. Respond with 'Saved' to when your blog are saved.
    """
)

In [107]:
write_agent = AssistantAgent(
    name="write_agent",
    model_client=az_model_client,
    system_message="""
        You are a blog writer, please help me write a blog based on news feed topics."
    """
)


In [108]:
text_termination = TextMentionTermination("Saved")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(10)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

In [109]:

reflection_team = RoundRobinGroupChat([news_feed_agent, write_agent, save_blog_content_agent], termination_condition=termination)


In [110]:
await Console(
                reflection_team.run_stream(task="""

                   I am writing a blog about Tesla. Find news feed for the following 3 questions and write a blog based on the search results ,save it
                    
                    1. How Tesla is performing?
                    2. Reliability of Tesla cars
                    3. Tesla's future plans                          

    """)
)  # Stream the messages to the console.

---------- user ----------


                   I am writing a blog about Tesla. Find news feed for the following 3 questions and write a blog based on the search results ,save it

                    1. How Tesla is performing?
                    2. Reliability of Tesla cars
                    3. Tesla's future plans                          

    
This is Code Interpreter for Azure AI Agent Service .......
Tesla performance
Created agent, ID: asst_5j21rjfQDc2czl0cKQ9C494Y
Created thread, ID: thread_oSeksTrJYyA7E529rvOSSGPg
SMS: {'id': 'msg_LYTln5zddijppPYFZ0SIkvDU', 'object': 'thread.message', 'created_at': 1743966508, 'assistant_id': None, 'thread_id': 'thread_oSeksTrJYyA7E529rvOSSGPg', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Hyundai’s Ioniq 6 redesign is a little sportier and a bit weirder - https://www.theverge.com/news/642799/hyundai-ioniq-6-2026-facelift-redesign\nSen. Mark Kelly is looking into selling his Tesla over Elon Musk. Other bi

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content="\n\n                   I am writing a blog about Tesla. Find news feed for the following 3 questions and write a blog based on the search results ,save it\n\n                    1. How Tesla is performing?\n                    2. Reliability of Tesla cars\n                    3. Tesla's future plans                          \n\n    ", type='TextMessage'), ToolCallRequestEvent(source='news_feed_agent', models_usage=RequestUsage(prompt_tokens=111, completion_tokens=16), metadata={}, content=[FunctionCall(id='call_C39KlE65TKS1J6mBwbpPPAk1', arguments='{"topic":"Tesla performance"}', name='generate_news_feed')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='news_feed_agent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content="Here is a summary of the recent news feeds:\n\n1. **Hyundai Ioniq 6 Redesign (2026)** - Hyundai's Ioniq 6 has received a facelift that introdu